Este tutorial utiliza modelos LSTM para predecir el nivel de emergencia de la estación Quintero (k540) en los intervalos 0-8, 8-16 y 16-24 horas siguientes. Para poder ejecutar este tutorial necesitas cumplir las siguientes condiciones:

* Modelos: se requiere tener los modelos ya entrenados para cada escenario. Para entrenar tus propios modelos, debes ejecutar `run_experiment.sh` con las configuraciones que más te interesen. De todas formas, en la carpeta *models* se brinda los modelos listos para ser utilizados.


* Testing set: se requiere el testing set para realizar las evaluaciones del modelo. Al igual que en la condición anterior, para generar tus propios testing set con nuevos datos, debes ejecutar `generator_inference_file.sh`.

Para continuar con el tutorial, los niveles de emergencia se categorizarán en 3 niveles dependiendo de las mediciones asociadas a la concentración del SO2, mas información respecto a la problemática y como se llevo a cabo el análisis recomendamos leer nuestro trabajo de investigación.

Antes de ejecutar el código, recomendamos generar el ambiente virtual con el archivo `environment_tf_keras.yml`.

# Paquetes y funciones

Cargamos los distintos paquetes a utilizar.

In [11]:
import numpy as np
import pandas as pd
import os, sys, glob

import tensorflow as tf

import pickle
import json
import random

from src.nowcastlib.dnn_models_f1 import build_categorical_model
from src.nowcastlib.data_handlers import MaxCategorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

La función `configure_model()` es la encargada de reconstruir el modelo a través de los distintos archivos de configuración, referentes al conjunto de datos (*ds_config*), entrenamiento (*tr_config*) y modelo (*md_config*). 

In [12]:
def configure_model(ds_config, tr_config, md_config):

    ds_config = tr_config['dataset']
    training_fields = ds_config['training_fields']
    n_variables = len(training_fields)
    
    number_class = int(md_config['n_class'])
    if number_class == 2:
        ranges = md_config['normalized_ranges_2class']
    elif number_class == 3:
        ranges = md_config['normalized_ranges_3class']
    elif number_class == 4:
        ranges = md_config['normalized_ranges_4class']
    else:
        raise ValueError("number of classes is not supported")

    lag = int(md_config['lag'])
    boost = int(md_config['boost'])
    lstm_neurons = int(md_config['lstm_neurons'])
    fcn_neurons = int(md_config['fcn_neurons'])
    fcn_layers = int(md_config['fcn_layers'])
    fcn_dropout = float(md_config['dropout'])
    
    train_config = tr_config['training']
    n_epochs = int(train_config['number_of_epochs'])
    batch_size = int(train_config['batch_size'])
    learning_rate = float(train_config['learning_rate'])
    beta_1 = float(train_config['beta_1'])
    beta_2 = float(train_config['beta_2'])

    # define model 
    compiled_model = build_categorical_model(lag, n_variables, number_class, lstm_neurons, fcn_neurons, fcn_layers, 
    fcn_dropout, learning_rate, beta_1, beta_2)
    
    compiled_model.summary()
    return compiled_model

# Lecturas archivos de configuración

Como especificaba la función anterior, necesitamos los distintos archivos de configuración para reconstruir el modelo, por lo que definimos la ruta de estos archivos.

In [13]:
ds_conf_path = 'config/sma_uai_alldata.json'
tr_conf_path = 'config/training_config.json'
md_conf_path = 'config/model_config.json'

Luego, creamos una configuración maestra que contemple todos los archivos de configuración.

In [14]:
master_config = dict()
for path in [ds_conf_path, tr_conf_path, md_conf_path]:
    config_file = open(path, 'r')
    config = json.loads(config_file.read())
    config_file.close()
    master_config.update(config)

Tras esto, desde la configuración maestra guardamos las configuraciones asociadas a los datos, entrenamiento y modelo sobre distintos objetos. 

In [15]:
ds_conf = master_config["dataset"]
tr_conf = master_config["training"]
md_conf = master_config["model"]

# Predicciones sobre ventana de tiempo 0-8 horas

Especificamos la estación a predecir, además del número de clases a considerar y la ventana de predicción. 

In [16]:
estacion = 'k540'
n_class = 3
boost = 8

Reemplazamos los valores definidos anteriormente en las configuraciones del modelo y entrenamiento

In [17]:
md_conf['boost'] = str(boost)
md_conf['n_class'] = str(n_class) 
tr_conf['dataset']['target_field'] = str(estacion) + '_so2_ppb'

Ahora cargamos el testing set, presentes en la carpeta *inference_files*.

In [18]:
outfile = '_inference_c' + str(n_class) + '_h' + str(boost) + '.pkl'

with open('inference_files/testx' + outfile, 'rb') as handle:
    test_x = pickle.load(handle)
with open('inference_files/testy' + outfile, 'rb') as handle:
    test_y = pickle.load(handle)

Si observamos las dimensiones del testing set, se tiene 6415 muestras. Para el caso de test_x, cada muestra posee 18 horas de información considerando 185 variables, mientras que test_y posee el nivel de emergencia asociada a cada muestra.

In [19]:
print(test_x.shape, test_y.shape)

(6415, 18, 185) (6415, 3)


Reconstruimos el modelo con la función `configure_model()`, además de añadir los pesos del modelo ya entrenado

In [20]:
ncast_model = configure_model(ds_conf, tr_conf, md_conf)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 18, 185)]         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 18, 128)           160768    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               6579

In [11]:
weights_filename = 'models/model_' + str(estacion) + '_' + str(n_class) + 'class_' + str(boost) + 'boost.h5'
ncast_model.load_weights(weights_filename)

Realizamos las predicciones sobre el test_x.

In [12]:
pred_test = ncast_model.predict(test_x)

Observamos las dimensiones, tanto para las predicciones como los valores reales

In [13]:
print(pred_test.shape, test_y.shape)

(6415, 3) (6415, 3)


Procedemos a evaluar el rendimiento del modelo sobre el test_y. En primer lugar, creamos la respectiva matriz de confusión

In [14]:
labels = ['bueno', 'regular', 'grave']
pd.DataFrame(confusion_matrix(np.argmax(test_y, axis=1), np.argmax(pred_test, axis=1), labels = [0,1,2]), index=labels, columns=labels)

,bueno,regular,grave
bueno,5816,112,299
regular,80,15,31
grave,31,10,21


Ahora, generemos el reporte de clasificación.

In [15]:
report = classification_report(test_y.argmax(axis=1), pred_test.argmax(axis=1))
str_rep = "{}".format(report)
print(str_rep)

              precision    recall  f1-score   support

           0       0.98      0.93      0.96      6227
           1       0.11      0.12      0.11       126
           2       0.06      0.34      0.10        62

    accuracy                           0.91      6415
   macro avg       0.38      0.46      0.39      6415
weighted avg       0.96      0.91      0.93      6415



# Predicciones sobre ventana de tiempo 8-16 horas

Anteriormente, realizamos predicciones para las primeras 8 horas. Entonces replicaremos el proceso anterior pero considerando la siguiente ventana de predicción, es decir, las 8-16 horas siguientes.

In [16]:
estacion = 'k540'
n_class = 3
boost = 16

Cargamos el testing set para este nueva ventana de predicción, además de reconstruir el modelo con sus respectivos pesos.

In [17]:
outfile = '_inference_c' + str(n_class) + '_h' + str(boost) + '.pkl'

with open('inference_files/testx' + outfile, 'rb') as handle:
    test_x = pickle.load(handle)
with open('inference_files/testy' + outfile, 'rb') as handle:
    test_y = pickle.load(handle)

ncast_model = configure_model(ds_conf, tr_conf, md_conf)

weights_filename = 'models/model_' + str(estacion) + '_' + str(n_class) + 'class_' + str(boost) + 'boost.h5'
ncast_model.load_weights(weights_filename)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 18, 185)]         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 18, 128)           160768    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               6579

Realizamos las predicciones sobre el test_x.

In [18]:
pred_test = ncast_model.predict(test_x)

Evaluamos el rendimiento observando su matriz de confusión y generando el reporte de clasificación.

In [19]:
labels = ['bueno', 'regular', 'grave']
pd.DataFrame(confusion_matrix(np.argmax(test_y, axis=1), np.argmax(pred_test, axis=1), labels = [0,1,2]), index=labels, columns=labels)

,bueno,regular,grave
bueno,5971,92,103
regular,97,8,21
grave,38,7,21


In [20]:
report = classification_report(test_y.argmax(axis=1), pred_test.argmax(axis=1))
str_rep = "{}".format(report)
print(str_rep)

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      6166
           1       0.07      0.06      0.07       126
           2       0.14      0.32      0.20        66

    accuracy                           0.94      6358
   macro avg       0.40      0.45      0.41      6358
weighted avg       0.95      0.94      0.95      6358



# Predicciones sobre ventana de tiempo 16-24 horas

Por último, realizamos predicciones sobre la ventana restante de 16-24 horas, así abarcamos el comportamiento de los niveles de emergencia por día siguiente.

In [21]:
estacion = 'k540'
n_class = 3
boost = 24

Cargamos el respectivo testing set, además de reconstruir el modelo con sus pesos.

In [22]:
outfile = '_inference_c' + str(n_class) + '_h' + str(boost) + '.pkl'

with open('inference_files/testx' + outfile, 'rb') as handle:
    test_x = pickle.load(handle)
with open('inference_files/testy' + outfile, 'rb') as handle:
    test_y = pickle.load(handle)

ncast_model = configure_model(ds_conf, tr_conf, md_conf)

weights_filename = 'models/model_' + str(estacion) + '_' + str(n_class) + 'class_' + str(boost) + 'boost.h5'
ncast_model.load_weights(weights_filename)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 18, 185)]         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 18, 128)           160768    
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               6579

Realizamos las predicciones sobre el test_x

In [23]:
pred_test = ncast_model.predict(test_x)

Evaluamos el rendimiento observando su matriz de confusión y generando el reporte de clasificación.

In [24]:
labels = ['bueno', 'regular', 'grave']
pd.DataFrame(confusion_matrix(np.argmax(test_y, axis=1), np.argmax(pred_test, axis=1), labels = [0,1,2]), index=labels, columns=labels)

,bueno,regular,grave
bueno,5770,169,163
regular,90,27,9
grave,31,15,20


In [26]:
report = classification_report(test_y.argmax(axis=1), pred_test.argmax(axis=1))
str_rep = "{}".format(report)
print(str_rep)

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      6102
           1       0.13      0.21      0.16       126
           2       0.10      0.30      0.16        66

    accuracy                           0.92      6294
   macro avg       0.40      0.49      0.43      6294
weighted avg       0.95      0.92      0.94      6294

